In [36]:
costs = np.array([-20,-30,-45])
fitness = 1 / (costs + 1e-10)  # prevencia delenia nulou
probabilities = fitness / np.sum(fitness)
probabilities

array([0.47368421, 0.31578947, 0.21052632])

In [48]:
import numpy as np
from typing import List, Tuple

class MKPSolverV3:
    def __init__(self, 
                 values: List[int],            
                 weights: List[List[int]],    
                 capacities: List[int],      
                 temperature: float = 1000.0, 
                 cooling_rate: float = 0.95,  
                 iterations: int = 100,     
                 min_temp: float = 1.0):   
        
        self.values = np.array(values)
        self.weights = np.array(weights)
        self.capacities = np.array(capacities)
        self.n_items = len(values)
        self.n_dimensions = len(capacities)
        
        self.temperature = temperature
        self.cooling_rate = cooling_rate
        self.iterations = iterations
        self.min_temp = min_temp
        
    def check_constraints(self, solution: np.ndarray) -> bool:
        for dim in range(self.n_dimensions):
            if np.sum(solution * self.weights[dim]) > self.capacities[dim]:
                return False
        return True
    
    def calculate_cost(self, solution: np.ndarray) -> float:
        if not self.check_constraints(solution):
            return float('-inf')
        return np.sum(solution * self.values)
    
    def generate_neighbor(self, solution: np.ndarray) -> np.ndarray:
        neighbor = solution.copy()
        if np.random.random() < 0.5:
            idx = np.random.randint(0, self.n_items)
            neighbor[idx] = 1 - neighbor[idx]
        else:
            idx1, idx2 = np.random.choice(self.n_items, 2, replace=False)
            neighbor[idx1] = 1 - neighbor[idx1]
            neighbor[idx2] = 1 - neighbor[idx2]
        return neighbor
    
    def solve(self) -> Tuple[np.ndarray, float]:
        current_solution = np.random.randint(2, size=self.n_items)
        while not self.check_constraints(current_solution):
            current_solution = np.random.randint(2, size=self.n_items)
        
        current_value = self.calculate_cost(current_solution)
        best_solution = current_solution.copy()
        best_value = current_value
        
        T = self.temperature
        
        while T > self.min_temp:
            neighbor_solution = self.generate_neighbor(current_solution)
            neighbor_value = self.calculate_cost(neighbor_solution)
                
            delta = neighbor_value - current_value
                
            if delta > 0 or np.random.random() < np.exp(delta / T):
                current_solution = neighbor_solution
                current_value = neighbor_value
                    
                if current_value > best_value:
                    best_solution = current_solution.copy()
                    best_value = current_value
            
            T *= self.cooling_rate
            
        return best_solution, best_value

class MKPSolverV2:
    def __init__(self, 
                 values: List[int],           
                 weights: List[List[int]],   
                 capacities: List[int],       
                 initial_temp: float = 100.0,
                 alpha: float = 0.95,       
                 max_iterations: int = 50,  
                 nrep: int = 100):          
        
        self.values = np.array(values)
        self.weights = np.array(weights)
        self.capacities = np.array(capacities)
        self.n_items = len(values)
        self.n_dimensions = len(capacities)
        
        self.T = initial_temp
        self.alpha = alpha
        self.max_iterations = max_iterations
        self.nrep = nrep
    
    def check_constraints(self, solution: np.ndarray) -> bool:
        for dim in range(self.n_dimensions):
            if np.sum(solution * self.weights[dim]) > self.capacities[dim]:
                return False
        return True
    
    def calculate_cost(self, solution: np.ndarray) -> float:
        if not self.check_constraints(solution):
            return float('-inf')
        return np.sum(solution * self.values)
    
    def get_neighbor(self, solution: np.ndarray) -> np.ndarray:
        neighbor = solution.copy()
        idx = np.random.randint(0, self.n_items)
        neighbor[idx] = 1 - neighbor[idx]
        
        if np.random.random() < 0.3:
            idx2 = np.random.randint(0, self.n_items)
            while idx2 == idx:  # vyhneme sa tomu istému predmetu
                idx2 = np.random.randint(0, self.n_items)
            neighbor[idx2] = 1 - neighbor[idx2]
            
        return neighbor

    def solve(self) -> Tuple[np.ndarray, float]:
        current_solution = np.zeros(self.n_items, dtype=int)
        while True:
            indices = np.random.choice(self.n_items, 
                                     size=np.random.randint(1, self.n_items//2), 
                                     replace=False)
            current_solution[indices] = 1
            if self.check_constraints(current_solution):
                break
            current_solution = np.zeros(self.n_items, dtype=int)

        current_cost = self.calculate_cost(current_solution)
        best_solution = current_solution.copy()
        best_cost = current_cost
        
        cool_iteration = 0
        
        while cool_iteration <= self.max_iterations:
            cool_iteration += 1
            temp_iteration = 0
            
            while temp_iteration <= self.nrep:
                temp_iteration += 1
                
                neighbor_solution = self.get_neighbor(current_solution)
                neighbor_cost = self.calculate_cost(neighbor_solution)
                
                delta = neighbor_cost - current_cost
                
                if delta > 0 or np.random.random() < np.exp(delta / self.T):
                    current_solution = neighbor_solution
                    current_cost = neighbor_cost
                    
                    if current_cost > best_cost:
                        best_solution = current_solution.copy()
                        best_cost = current_cost
            
            self.T = self.alpha * self.T
            
        return best_solution, best_cost

class MKPSolverV1:
    def __init__(self, 
                 values: List[int],
                 weights: List[List[int]],
                 capacities: List[int],
                 population_size: int = 50,
                 cooling_rate: float = 0.95,
                 mutation_rate: float = 0.1,
                 crossover_rate: float = 0.8):
        
        self.values = np.array(values)
        self.weights = np.array(weights)
        self.capacities = np.array(capacities)
        self.n_items = len(values)
        self.n_dimensions = len(capacities)
        
        self.population_size = population_size
        self.cooling_rate = cooling_rate
        self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate
        
    def check_constraints(self, solution: np.ndarray) -> bool:
        for dim in range(self.n_dimensions):
            if np.sum(solution * self.weights[dim]) > self.capacities[dim]:
                return False
        return True
    
    def calculate_cost(self, solution: np.ndarray) -> float:
        if not self.check_constraints(solution):
            return float('inf')
        return -np.sum(solution * self.values)
    
    def generate_initial_population(self) -> np.ndarray:
        population = np.zeros((self.population_size, self.n_items), dtype=int)
        for i in range(self.population_size):
            while True:
                solution = np.random.randint(2, size=self.n_items)
                if self.check_constraints(solution):
                    population[i] = solution
                    break
        return population
    
    def crossover(self, parent1: np.ndarray, parent2: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        if np.random.random() < self.crossover_rate:
            crossover_point = np.random.randint(1, self.n_items)
            offspring1 = np.concatenate([parent1[:crossover_point], parent2[crossover_point:]])
            offspring2 = np.concatenate([parent2[:crossover_point], parent1[crossover_point:]])
            return offspring1, offspring2
        return parent1.copy(), parent2.copy()
    
    def mutate(self, solution: np.ndarray) -> np.ndarray:
        for i in range(self.n_items):
            if np.random.random() < self.mutation_rate:
                solution[i] = 1 - solution[i]
        return solution
    
    def apply_genetic_operators(self, population: np.ndarray) -> np.ndarray:
        costs = np.array([self.calculate_cost(solution) for solution in population])
        fitness = 1 / (costs - 1e-10)  # prevencia delenia nulou
        probabilities = fitness / np.sum(fitness)
        
        new_population = np.zeros_like(population)
        for i in range(0, self.population_size, 2):
            parents_idx = np.random.choice(self.population_size, size=2, p=probabilities)
            parent1, parent2 = population[parents_idx]
            
            offspring1, offspring2 = self.crossover(parent1, parent2)
            offspring1 = self.mutate(offspring1)
            offspring2 = self.mutate(offspring2)
            
            new_population[i] = offspring1
            if i + 1 < self.population_size:
                new_population[i + 1] = offspring2
                
        return new_population
    
    def local_search(self, solution: np.ndarray) -> np.ndarray:
        improved = True
        while improved:
            improved = False
            best_neighbor = solution.copy()
            best_cost = self.calculate_cost(solution)
            
            for i in range(self.n_items):
                neighbor = solution.copy()
                neighbor[i] = 1 - neighbor[i]
                
                if self.check_constraints(neighbor):
                    cost = self.calculate_cost(neighbor)
                    if cost < best_cost:
                        best_neighbor = neighbor
                        best_cost = cost
                        improved = True
            
            if improved:
                solution = best_neighbor
                
        return solution
    
    def solve(self) -> Tuple[np.ndarray, float]:
        population = self.generate_initial_population()
        
        costs = np.array([self.calculate_cost(solution) for solution in population])
        best_solution = population[np.argmin(costs)]
        best_cost = np.min(costs)
        
        T = (np.max(costs) - np.min(costs)) / (self.population_size / 2)
        
        frozen = False
        generation = 0
        max_generations = 100
        
        while not frozen:
            new_population = np.zeros_like(population)
            current_point = best_solution
            
            for i in range(self.population_size):
                next_point = self.mutate(current_point.copy())
                
                if self.check_constraints(next_point):
                    delta_cost = self.calculate_cost(next_point) - self.calculate_cost(current_point)
                    
                    if delta_cost < 0 or np.random.random() < np.exp(-delta_cost / T):
                        new_population[i] = next_point
                        current_point = next_point
                    else:
                        new_population[i] = population[np.random.randint(self.population_size)]
            
            population = self.apply_genetic_operators(new_population)
            
            costs = np.array([self.calculate_cost(solution) for solution in population])
            current_best = population[np.argmin(costs)]
            current_best_cost = np.min(costs)
            
            if current_best_cost < best_cost:
                best_solution = current_best
                best_cost = current_best_cost
            
            T *= self.cooling_rate
            generation += 1
            
            if generation >= max_generations or T < 0.01:
                frozen = True
        
        best_solution = self.local_search(best_solution)
        best_cost = self.calculate_cost(best_solution)
        
        return best_solution, -best_cost

def test_mkp_solvers():
    scenarios = [
    {
        "values": [10, 15, 20, 25, 30],
        "weights": [
            [2, 3, 4, 5, 6],  
            [3, 4, 5, 6, 7]
        ],
        "capacities": [10, 12]
    },
    {
        "values": [12, 7, 11, 8, 9, 20, 15, 25, 14, 30],
        "weights": [
            [3, 4, 5, 6, 7, 8, 4, 5, 6, 7],
            [4, 3, 6, 7, 8, 9, 5, 4, 8, 6],
            [5, 6, 4, 5, 7, 6, 4, 8, 5, 9]
        ],
        "capacities": [15, 20, 18]
    },
    {
        "values": [20, 30, 10, 40, 50, 25, 35, 45, 15, 60],
        "weights": [
            [4, 5, 3, 6, 8, 6, 7, 9, 4, 5],
            [5, 6, 4, 7, 5, 8, 6, 5, 7, 10],
            [6, 7, 5, 9, 6, 4, 8, 7, 5, 6]
        ],
        "capacities": [30, 35, 28]
    },
    {
        "values": [10, 15, 20, 25, 30, 35, 40, 45, 50],
        "weights": [
            [6, 7, 8, 3, 5, 7, 8, 6, 9],
            [4, 5, 6, 7, 8, 5, 4, 6, 5],
            [5, 3, 5, 6, 7, 4, 8, 5, 6]
        ],
        "capacities": [20, 25, 22]
    },
    {
        "values": [55, 35, 25, 40, 45, 50, 30, 60, 20, 65],
        "weights": [
            [7, 5, 4, 6, 8, 6, 5, 9, 7, 8],
            [4, 6, 3, 7, 6, 4, 7, 5, 6, 7],
            [5, 8, 6, 7, 9, 5, 4, 8, 7, 9],
            [6, 7, 5, 8, 6, 4, 7, 6, 9, 10]
        ],
        "capacities": [25, 30, 32, 28]
    },
    {
        "values": [25, 20, 30, 15, 35, 45, 10, 50, 55, 60, 40, 25, 30],
        "weights": [
            [8, 5, 7, 6, 9, 8, 6, 10, 5, 8, 7, 6, 9],
            [7, 6, 8, 5, 9, 7, 5, 6, 8, 7, 5, 4, 6],
            [9, 8, 6, 7, 10, 9, 5, 7, 6, 8, 9, 5, 6],
            [6, 5, 7, 8, 9, 7, 6, 10, 8, 9, 5, 7, 6]
        ],
        "capacities": [35, 40, 38, 32]
    }
    ]

    
    # Definícia algoritmov
    solvers = [
        ("MKPSolverV1", lambda values, weights, capacities: MKPSolverV1(
            values=values,
            weights=weights,
            capacities=capacities,
            population_size=50,
            cooling_rate=0.95,
            mutation_rate=0.1,
            crossover_rate=0.8
        )),
        ("MKPSolverV2", lambda values, weights, capacities: MKPSolverV2(
            values=values,
            weights=weights,
            capacities=capacities,
            initial_temp=100.0,
            alpha=0.95,
            max_iterations=50,
            nrep=100
        )),
        ("MKPSolverV3", lambda values, weights, capacities: MKPSolverV3(
            values=values,
            weights=weights,
            capacities=capacities,
            temperature=1000.0,
            cooling_rate=0.95,
            iterations=100,
            min_temp=1.0
        ))
    ]
    
    for idx, scenario in enumerate(scenarios):
        print(f"\nScenár {idx + 1}:")
        best_value = -np.inf
        best_solution = None
        best_solver = None
        
        for solver_name, solver_fn in solvers:
            solver = solver_fn(scenario["values"], scenario["weights"], scenario["capacities"])
            solution, value = solver.solve()
            
            print(f"\nAlgoritmus: {solver_name}")
            print(f"  Najlepšie riešenie: {solution}")
            print(f"  Hodnota riešenia: {value}")
            print("  Využitie kapacít:")
            for dim in range(len(scenario["capacities"])):
                used_capacity = np.sum(solution * scenario["weights"][dim])
                print(f"    Dimenzia {dim+1}: {used_capacity}/{scenario['capacities'][dim]}")
            
            if value > best_value:
                best_value = value
                best_solution = solution
                best_solver = solver_name
        
        print(f"\nNajlepší algoritmus pre Scenár {idx + 1}: {best_solver}")
        print(f"  Najlepšia hodnota riešenia: {best_value}")
        print(f"  Najlepšie riešenie: {best_solution}")

# def example_mkp():
#     values = [10, 15, 20, 25, 30]
#     weights = [
#         [2, 3, 4, 5, 6],  
#         [3, 4, 5, 6, 7]  
#     ]
#     capacities = [10, 12] 
    
#     values = [12, 7, 11, 8, 9, 20, 15, 25, 14, 30]
#     weights = [
#         [3, 4, 5, 6, 7, 8, 4, 5, 6, 7],   # Dimension 1 (weight in first constraint)
#         [4, 3, 6, 7, 8, 9, 5, 4, 8, 6],   # Dimension 2 (weight in second constraint)
#         [5, 6, 4, 5, 7, 6, 4, 8, 5, 9],   # Dimension 3 (weight in third constraint)
#     ]
#     capacities = [15, 20, 18]
#     # solver = MKPSolverV3(
#     #     values=values,
#     #     weights=weights,
#     #     capacities=capacities,
#     #     temperature=1000.0,
#     #     cooling_rate=0.95,
#     #     iterations=100,
#     #     min_temp=1.0
#     # )

#     # solver = MKPSolverV2(
#     #     values=values,
#     #     weights=weights,
#     #     capacities=capacities,
#     #     initial_temp=100.0,
#     #     alpha=0.95,
#     #     max_iterations=50,
#     #     nrep=100
#     # )

#     # solver = MKPSolverV1(
#     #     values=values,
#     #     weights=weights,
#     #     capacities=capacities,
#     #     population_size=50,
#     #     cooling_rate=0.95,
#     #     mutation_rate=0.1,
#     #     crossover_rate=0.8
#     # )
    
#     solution, value = solver.solve()
    
#     print(f"Najlepšie riešenie: {solution}")
#     print(f"Hodnota riešenia: {value}")
#     print(f"Využitie kapacít:")
#     for dim in range(len(capacities)):
#         used_capacity = np.sum(solution * weights[dim])
#         print(f"Dimenzia {dim+1}: {used_capacity}/{capacities[dim]}")

if __name__ == "__main__":
    test_mkp_solvers()


Scenár 1:

Algoritmus: MKPSolverV1
  Najlepšie riešenie: [0 0 1 0 1]
  Hodnota riešenia: 50
  Využitie kapacít:
    Dimenzia 1: 10/10
    Dimenzia 2: 12/12

Algoritmus: MKPSolverV2
  Najlepšie riešenie: [0 0 1 0 1]
  Hodnota riešenia: 50
  Využitie kapacít:
    Dimenzia 1: 10/10
    Dimenzia 2: 12/12

Algoritmus: MKPSolverV3
  Najlepšie riešenie: [0 0 1 0 1]
  Hodnota riešenia: 50
  Využitie kapacít:
    Dimenzia 1: 10/10
    Dimenzia 2: 12/12

Najlepší algoritmus pre Scenár 1: MKPSolverV1
  Najlepšia hodnota riešenia: 50
  Najlepšie riešenie: [0 0 1 0 1]

Scenár 2:

Algoritmus: MKPSolverV1
  Najlepšie riešenie: [1 0 0 0 0 0 1 0 0 1]
  Hodnota riešenia: 57
  Využitie kapacít:
    Dimenzia 1: 14/15
    Dimenzia 2: 15/20
    Dimenzia 3: 18/18

Algoritmus: MKPSolverV2
  Najlepšie riešenie: [1 0 0 0 0 0 1 0 0 1]
  Hodnota riešenia: 57
  Využitie kapacít:
    Dimenzia 1: 14/15
    Dimenzia 2: 15/20
    Dimenzia 3: 18/18

Algoritmus: MKPSolverV3
  Najlepšie riešenie: [1 0 0 0 0 0 0 1 1 0]
 